In [ ]:
import os
import pandas as pd
from PyEMD import EMD
import numpy as np

labels = []
all_files = []
path = 'D:/vibration'
labels_list = os.listdir(path)
print(labels_list)        
for i in labels_list:
    wavs_path = os.path.join(path,i)
    second_dirs = os.listdir(wavs_path)
    for csv in second_dirs:
        labels.append(i)
        csvFilePath = wavs_path+'/'+csv
        all_files.append(csvFilePath)
print(labels)
print(all_files)

In [ ]:
import torch
from torch.utils.data import Dataset

from vmdpy import VMD
import numpy as np


import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import os

path = 'D:/vibration'
class MyDataset(Dataset):
    def __init__(self):
        labels = []
        path = 'D:/vibration'
        labels_list = os.listdir(path)
        for i in labels_list:
            wavs_path = os.path.join(path, i)
            second_dirs = os.listdir(wavs_path)
            for csv in second_dirs:
                labels.append(i)
        self.label = np.array(labels)
        # 创建 OneHotEncoder 对象
        self.encoder = OneHotEncoder(sparse_output=False)
        # 对标签进行独热编码
        encoded_labels = self.encoder.fit_transform([[label] for label in self.label])
        # 将独热编码结果保存为 Tensor
        self.encoded_labels_tensor = torch.tensor(encoded_labels, dtype=torch.float32)

    def __len__(self):
        return len(self.encoded_labels_tensor)

    def __getitem__(self,index):
        Fs=1024 # 采样频率
        N=1024 # 采样点数
        t=np.arange(1,N+1)/N
        fre_axis=np.linspace(0,Fs/2,int(N/2))
        label = self.encoded_labels_tensor[index]
        alpha=2000
        tau=0 # tau 噪声容限，即允许重构后的信号与原始信号有差别。
        K=3 # K 分解模态（IMF）个数
        DC=0 # DC 若为0则让第一个IMF为直流分量/趋势向量
        init=1 # init 指每个IMF的中心频率进行初始化。当初始化为1时，进行均匀初始化。
        tol=1e-7 # 控制误差大小常量，决定精度与迭代次数
        all_files = []
        path = 'D:/vibration'
        labels_list = os.listdir(path)
        for i in labels_list:
            wavs_path = os.path.join(path,i)
            second_dirs = os.listdir(wavs_path)
            for csv in second_dirs:
                csvFilePath = wavs_path+'/'+csv
                all_files.append(csvFilePath)
        csvpath = all_files[index]
        data = pd.read_csv(csvpath)
        signal = data['0']
        signal = np.array(signal)
        u, u_hat, omega = VMD(signal, alpha, tau, K, DC, init, tol) # 输出U是各个IMF分量，u_hat是各IMF的频谱，omega为各IMF的中心频率
        # 返回加载和预处理后的数据
        return torch.as_tensor(u,dtype=torch.float32), torch.as_tensor(label,dtype=torch.float)



In [ ]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

In [ ]:
dataset1 = MyDataset()
total_size = len(dataset1)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size
print(total_size,train_size,val_size,test_size)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset1, [train_size, val_size, test_size])


In [ ]:
data,label = dataset1[8422]
print(data.shape)
print(label)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim,num_classes,num_layers, num_heads, dropout,linear_dim1,linear_dim2):
        super(TransformerClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, input_dim)
        self.position_encoding = PositionalEncoding(hidden_dim)
        self.transformer_encoder = TransformerEncoder(hidden_dim, num_layers, num_heads, dropout)
        self.MLPclassifier = MLPClassifier(hidden_dim,linear_dim1,linear_dim2,num_classes)

    def forward(self, x):
        x = self.linear(x)
        encoded = self.position_encoding(x)
        transformed = self.transformer_encoder(encoded)
        output = self.MLPclassifier(transformed)
        return output


class PositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_length=3):
        super(PositionalEncoding, self).__init__()

        position_encoding = torch.zeros(max_length, hidden_dim)
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-math.log(10000.0) / hidden_dim))

        position_encoding[:, 0::2] = torch.sin(position * div_term)
        position_encoding[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('position_encoding', position_encoding.unsqueeze(0))

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.position_encoding[:x.size(0), :]
        return x


class TransformerEncoder(nn.Module):
    def __init__(self, hidden_dim, num_layers, num_heads, dropout):
        super(TransformerEncoder, self).__init__()

        self.layers = nn.ModuleList([
            TransformerEncoderLayer(hidden_dim, num_heads, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class TransformerEncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_heads, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.multihead_attention = nn.MultiheadAttention(hidden_dim, num_heads, dropout=dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim)
        )
        self.layer_norm1 = nn.LayerNorm(hidden_dim)
        self.layer_norm2 = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attended = self.multihead_attention(x, x, x)[0]
        x = x + self.dropout(attended)
        x = self.layer_norm1(x)

        fed_forward = self.feed_forward(x)
        x = x + self.dropout(fed_forward)
        x = self.layer_norm2(x)

        return x

class MLPClassifier(nn.Module):
    def __init__(self,hidden_dim,linear_dim1,linear_dim2,num_classes):
        super(MLPClassifier, self).__init__()

        self.linear_1 = nn.Linear(hidden_dim,linear_dim1)
        self.linear_2 = nn.Linear(linear_dim1,linear_dim2)
        self.linear_3 = nn.Linear(linear_dim2,num_classes)

    
    def forward(self,x):
        x = self.linear_1(x)
        x = F.relu(x)
        x = self.linear_2(x)
        x = F.relu(x)
        x = self.linear_3(x)
        x = F.softmax(x,dim=1)
        
        return x


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 创建模型实例
model = TransformerClassifier(input_dim=1024,
                              hidden_dim=1024,
                              num_classes=5,
                              num_layers=1,
                              num_heads=8,
                              linear_dim1=500,
                              linear_dim2=100,
                              dropout=0.1).cuda()
for param in model.parameters():
    print(type(param), param.size())
    
num_params = sum([param.nelement() for param in model.parameters()])
print(num_params)

In [ ]:
print(torch.cuda.is_available())
print(torch.version.cuda)
model = model.to('cuda')

for name, param in model.named_parameters():
    print(f"Parameter: {name}, Device: {param.device}")


In [ ]:
# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 设置训练参数
num_epochs = 5

# 训练循环
for epoch in range(num_epochs):
    model.train() # 将模型设置为训练模式
    running_loss = 0.0

    for inputs, labels in tqdm(train_dataloader):
        inputs, labels = inputs.cuda(),labels.cuda()
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


    # 输出每个epoch的训练损失
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_dataloader)}")

    # 验证循环
    model.eval() # 将模型设置为评估模式
    val_loss = 0.0
    correct_preds = 0
    total_preds = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_dataloader):
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels.long())
            val_loss += loss.item()

            _, predicted = torch.max(outputs,1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
        
        val_acc = correct_preds / total_preds

    # 输出每个epoch的验证损失和准确率
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss/len(val_dataloader)}, Accuracy: {val_acc}")